# Test dnnwrite.py 
 >__Created__:  2018 Harrison B. Prosper



In [1]:
import os, sys
import pandas as pd
import numpy as np

from pprint import PrettyPrinter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

import matplotlib as mp
import matplotlib.pyplot as plt
import pylab as pl
import ROOT

%matplotlib inline

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Welcome to JupyROOT 6.12/04


In [2]:
# update fonts
FONTSIZE = 24
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 24}

mp.rc('font', **font)

In [3]:
print ROOT.gSystem.Load('DNN/lib/libDNN')

0


In [4]:
dnn = ROOT.dnn_vbf_ggf()

### Load Higgs boson data into memory


In [5]:
train_data, test_data = joblib.load('../03/higgs_vbf_ggf_data.pkl')
train_data[['target','detajj', 'massjj']][:10]

,target,detajj,massjj
30257,1.0,0.7547,212.5
48940,1.0,5.1280,558.4
60286,1.0,4.8450,1012.0
14710,0.0,3.5240,797.4
26311,1.0,1.0550,107.3
2051,0.0,3.1290,680.2
26521,1.0,2.3110,189.6
34661,1.0,5.7930,681.6
33682,1.0,3.2240,249.3
49316,0.0,1.0170,538.8


### Load previously trained MLP

In [6]:
mlp, scaler = joblib.load("../03/03_dnn_higgs_vbf_ggf.pkl")
mlp, scaler

(MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(10, 10), learning_rate='constant',
        learning_rate_init=0.001, max_iter=500, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
        solver='adam', tol=1e-06, validation_fraction=0.1, verbose=0,
        warm_start=False),
 StandardScaler(copy=True, with_mean=True, with_std=True))

### Prepare data for training/testing

In [7]:
inputs  = ['detajj', 'massjj']
X_train = scaler.transform(train_data[inputs]); y_train = train_data['target']
X_test  = scaler.transform(test_data[inputs]); y_test = test_data['target']
print "mean: %s" % scaler.mean_
print "scale: %s" % scaler.scale_

mean: [   2.94192279  432.64474879]
scale: [   1.90830183  506.94648289]


### Check DNN class. It should give same output as MLP

In [8]:
yy = mlp.predict_proba(X_test)
print "%21s\t%10s\t%10s %10s" % ('inputs', 'target', 'DNN', 'MLP')

for i in range(20):
    target = test_data['target'].iloc[i]
    detajj = test_data['detajj'].iloc[i]
    massjj = test_data['massjj'].iloc[i]
    
    y1 = dnn((detajj, massjj))
    y2 = yy[i][1]
    
    print "%10.3f %10.3f\t%10d\t%10.4f %10.4f" % (detajj, massjj, target, y1, y2)


               inputs	    target	       DNN        MLP
     0.407    243.500	         1	    0.4125     0.4125
     4.655    676.700	         1	    0.7617     0.7617
     1.120     67.220	         1	    0.1863     0.1863
     0.374     19.000	         0	    0.1287     0.1287
     0.480     44.900	         0	    0.1752     0.1752
     0.488     71.080	         0	    0.2431     0.2431
     1.418    179.600	         0	    0.4202     0.4202
     0.025     60.400	         1	    0.2496     0.2496
     4.539    427.000	         1	    0.6676     0.6676
     6.143   1322.000	         1	    0.8167     0.8167
     5.603   1030.000	         1	    0.7975     0.7975
     5.423    633.700	         1	    0.6708     0.6708
     4.412    351.500	         0	    0.5263     0.5263
     3.521    596.300	         1	    0.7219     0.7219
     2.729    164.300	         0	    0.3135     0.3135
     1.263     69.320	         0	    0.1819     0.1819
     6.098   1064.000	         0	    0.7743     0.7743
     5.706